# visualizer.ipynb

In [1]:
import sqlite3
import cv2
import pandas as pd
import numpy as np
import os

In [12]:
def annotate_image_with_polygon_and_rectangle(original_image_path, x_min, y_min, x_max, y_max, polygon_points, color):
    # Load original image
    img_original = cv2.imread(original_image_path).copy()
    
    # Overlay detected object with a semitransparent polygon mask
    img_overlay = np.zeros_like(img_original)
    # mask_color = (0, 0, 255)  # Red color in BGR
    alpha = 0.4  # Transparency factor (0.0 = transparent, 1.0 = opaque)
    img_overlay = cv2.fillPoly(img_overlay, polygon_points, color)
    img_annotated = cv2.addWeighted(img_overlay, alpha, img_original, 1 - alpha, 0)

    # Draw rectangle
    start_point = (x_min, y_min)
    end_point = (x_max, y_max)
    thickness = 1
    cv2.rectangle(img_annotated, start_point, end_point, color, thickness)
    
    # Extract the region inside the rectangle
    roi = img_annotated[y_min:y_max+1, x_min:x_max+1]
    
    return img_annotated, roi

In [13]:
# Set variables for this run
database_path = 'sam3_detections.sqlite3'
sql = 'select * from detections order by image_path, confidence desc;'
annotated_image_dir = 'annotated_images'

# Ensure existence of output directory
os.makedirs(annotated_image_dir, exist_ok=True)

# Read detections from the database
conn = sqlite3.connect(database_path)
df = pd.read_sql_query(sql, conn)
conn.close()

# Annotate images based on detections (one image per detection)
# Images are saved as new files with -XX.jpg suffix where XX is the order returned by the SQL query.
for i, r in df.iterrows():
    original_image_path = r['image_path']
    x_min = int(r['x_min'])
    y_min = int(r['y_min'])
    x_max = int(r['x_max'])
    y_max = int(r['y_max'])
    polygon_points_str = r['polygon_points_str']
    polygon_points = np.array(eval(polygon_points_str), np.int32)          
    polygon_points = [polygon_points.reshape((-1, 1, 2))]
    color = (0, 0, 255)  # Red color in BGR
    annotated_image_path = f'{annotated_image_dir}/{original_image_path}-{i:02d}.jpg' 
    roi_image_path = f'{annotated_image_dir}/roi-{original_image_path}-{i:02d}.jpg' 
    print(f'Annotating image: {annotated_image_path}')   
    img_annotated, roi = annotate_image_with_polygon_and_rectangle(original_image_path, x_min, y_min, x_max, y_max, polygon_points, color)
    print(f'saving {annotated_image_path}')
    print(cv2.imwrite(annotated_image_path, img_annotated))
    print(f'saving {roi_image_path}')
    print(cv2.imwrite(roi_image_path, roi))
    
    # Write .txt file describing the annotated image. May be used in a caption later.
    txt_path = f'{annotated_image_path}.txt'
    with open(txt_path, 'w') as f:
        f.write(f'confidence: {r["confidence"]:.3f} object_index: {r["object_index"]}\n')
    
print('FINISHED')

Annotating image: annotated_images/08hs-palms-03-zglw-superJumbo.webp-00.jpg
saving annotated_images/08hs-palms-03-zglw-superJumbo.webp-00.jpg
True
saving annotated_images/roi-08hs-palms-03-zglw-superJumbo.webp-00.jpg
True
Annotating image: annotated_images/08hs-palms-03-zglw-superJumbo.webp-01.jpg
saving annotated_images/08hs-palms-03-zglw-superJumbo.webp-01.jpg
True
saving annotated_images/roi-08hs-palms-03-zglw-superJumbo.webp-01.jpg
True
Annotating image: annotated_images/20251129_152106.jpg-02.jpg
saving annotated_images/20251129_152106.jpg-02.jpg
True
saving annotated_images/roi-20251129_152106.jpg-02.jpg
True
Annotating image: annotated_images/20251129_152106.jpg-03.jpg
saving annotated_images/20251129_152106.jpg-03.jpg
True
saving annotated_images/roi-20251129_152106.jpg-03.jpg
True
Annotating image: annotated_images/20251129_152106.jpg-04.jpg
saving annotated_images/20251129_152106.jpg-04.jpg
True
saving annotated_images/roi-20251129_152106.jpg-04.jpg
True
Annotating image: an

In [14]:
cv2.imwrite('roi.jpg', roi)

True